<a href="https://colab.research.google.com/github/hawc2/wikidata/blob/main/Wikidata_SPARQL_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Querying and Visualizing Wikidata Overview




## Instructions

This notebook guides you through querying and visualizing wikidata.

# Install Packages

In [ ]:
!pip install SPARQLWrapper
%load_ext google.colab.data_table 
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

     |████████████████████████████████| 379 kB 11.2 MB/s 
     |████████████████████████████████| 45 kB 4.4 MB/s 


# Get Your Data

Using Sparqlwrapper Query Wikidata API

## Set up SPARQL Query

### Relevant Data

https://docs.google.com/spreadsheets/d/1grYsGIwp6yey0ZPdkFXJlzlwZSQ6-4mRHO2XoZYicSQ/edit#gid=0

### Potential Additional SPAQRL Properties

1) social media and website info

2) can we pull first and last name separately.  P735 and P734 for given name and family name


3) geolocating sparql

4) what about sexual orientation/identity?

5) tracing familial and professional connections; 'trained by, influenced by' family connections to other regions
regional cultural background

6) no field being used often for 'exhibiting' - 'has works in collection' is a consistent field, but not much individual gallery shows-events

7) are there other institutiosn that hold these artists? like detroit / michigan artists connections with philly? what connections do philly artists have with other cities, artists outside philly?

8) network connections - significant person field? captures people who were peers, colleagues, if people exhibited together - artists who collaborated together?

9) can we complicate how we're defining ethnicity? synatra has some other properties

In [ ]:
 sparql.setQuery("""
SELECT
    ?artist ?artistLabel ?sexGenderLabel ?birthdayLabel ?deathDateLabel
    (group_concat(DISTINCT(?occupationLabel);separator=", ") as ?occupations)
    (group_concat(DISTINCT(?residenceLabel);separator=", ") as ?residences)
    (group_concat(DISTINCT(?educationLabel);separator=", ") as ?educations)
    (group_concat(DISTINCT(?employerLabel);separator=", ") as ?employers)
    (group_concat(DISTINCT(?birthPlaceLabel);separator=", ") as ?birthPlaces)
    (group_concat(DISTINCT(?deathPlaceLabel);separator=", ") as ?deathPlaces)
    
WHERE
{
    ?artist wdt:P5008 wd:Q94124522. # PMA African American artists
    ?artist wdt:P106 ?occupation
    OPTIONAL { ?artist wdt:P21 ?sexGender. }
    OPTIONAL { ?artist wdt:P569 ?birthdayLabel. }
    OPTIONAL { ?artist wdt:P570 ?deathDateLabel. }
    OPTIONAL { ?artist wdt:P19 ?birthPlace. }
    OPTIONAL { ?artist wdt:P20 ?deathPlace. }
    OPTIONAL { ?artist wdt:P551 ?residence. }
    OPTIONAL { ?artist wdt:P69 ?education. }
    OPTIONAL { ?artist wdt:P108 ?employer. }
    SERVICE wikibase:label { 
    bd:serviceParam wikibase:language "en". 
    ?artist rdfs:label ?artistLabel . 
    ?occupation rdfs:label ?occupationLabel .
    ?sexGender rdfs:label ?sexGenderLabel .
    ?birthPlace rdfs:label ?birthPlaceLabel .
    ?deathPlace rdfs:label ?deathPlaceLabel .
    ?residence rdfs:label ?residenceLabel .
    ?education rdfs:label ?educationLabel .
    ?employer rdfs:label ?employerLabel .
  }
}
GROUP BY ?artist ?artistLabel ?sexGenderLabel ?birthdayLabel ?deathDateLabel
ORDER BY ?artistLabel
""")

In [ ]:
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


In [ ]:
results

{'head': {'vars': ['artist',
   'artistLabel',
   'sexGenderLabel',
   'birthdayLabel',
   'deathDateLabel',
   'occupations',
   'residences',
   'educations',
   'employers',
   'birthPlaces',
   'deathPlaces']},
 'results': {'bindings': [{'artist': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q89042076'},
    'artistLabel': {'type': 'literal',
     'value': 'A.J. Smith',
     'xml:lang': 'en'},
    'birthPlaces': {'type': 'literal', 'value': ''},
    'birthdayLabel': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
     'type': 'literal',
     'value': '1952-01-01T00:00:00Z'},
    'deathPlaces': {'type': 'literal', 'value': ''},
    'educations': {'type': 'literal', 'value': ''},
    'employers': {'type': 'literal',
     'value': 'University of Arkansas at Little Rock'},
    'occupations': {'type': 'literal',
     'value': 'professor, artist, printmaker'},
    'residences': {'type': 'literal', 'value': ''},
    'sexGenderLabel': {'type': 'literal', 'value':

# Wrangle your Data

1) Why do we have multiple records/rows for some artists? 
John Woodrow Wilson for instance
Concatenate more fields

2) standardize null values NaN or blank

3) birthdate/deathdate values could also be simplified and clarified, doesnt' look like time is worth keeping - *just simplify it to year*

4) set up geopandas to geolocate addresses

## Create Dataframe


In [ ]:
df = pd.io.json.json_normalize(results['results']['bindings'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


In [ ]:
cols = ['artistLabel.value', 
            'sexGenderLabel.value', 
            'birthdayLabel.value',
            'deathDateLabel.value',
            'birthPlaces.value',
            'deathPlaces.value',
            'residences.value',
            'educations.value',
            'occupations.value',
            'employers.value']

In [ ]:
df = df[cols]

In [ ]:
df.columns = ['name', 'gender', 'birthdate', 'deathdate', 'birthplace', 'deathplace', 'residence', 'education', 'occupation', 'employer']
df

,name,gender,birthdate,deathdate,birthplace,deathplace,residence,education,occupation,employer
0,A.J. Smith,male,1952-01-01T00:00:00Z,NaN,,,,,"professor, artist, printmaker",University of Arkansas at Little Rock
1,Aaron Douglas,male,1899-05-26T00:00:00Z,1979-02-02T00:00:00Z,Topeka,Nashville,,"University of Kansas, University of Nebraska–L...","illustrator, painter, muralist",Fisk University
2,Akili Ron Anderson,male,1946-02-19T00:00:00Z,NaN,"Washington, D.C.",,"Washington, D.C.",Howard University,"photographer, artist, painter, sculptor, stain...",
3,Alfred A. Smith,male,1896-09-17T00:00:00Z,1940-01-01T00:00:00Z,New York City,,"New York City, Paris",,artist,
4,Alison Saar,female,1956-02-05T00:00:00Z,NaN,Los Angeles,,,"Otis College of Art and Design, Scripps College","photographer, artist, illustrator, painter, sc...",
...,...,...,...,...,...,...,...,...,...,...
220,William Majors,male,1930-07-21T00:00:00Z,1982-08-29T00:00:00Z,Indianapolis,Portsmouth,,Herron School of Art and Design,"artist, university teacher",
221,William Plummer,male,1873-01-01T00:00:00Z,1943-01-01T00:00:00Z,,,Smyth County,,"inventor, cabinetmaker",
222,William T. Williams,male,1942-07-17T00:00:00Z,NaN,Cross Creek Township,,,"Yale University, Pratt Institute, Yale School ...","professor, artist, painter, printmaker",Brooklyn College
223,Willie Birch,male,1942-01-01T00:00:00Z,NaN,New Orleans,,New Orleans,"Southern University, Maryland Institute Colleg...","artist, university teacher",Touro College


In [ ]:
## Gathering duplicate entries so I can figure out which columns still need to be concatenated.
dupe = df[df.duplicated(subset=['name'], keep=False)]
dupe

,name,gender,birthdate,deathdate,birthplace,deathplace,residence,education,occupation,employer
17,Beauford Delaney,male,1901-12-30T00:00:00Z,1979-03-25T00:00:00Z,Knoxville,14th arrondissement of Paris,,"Harvard University, Austin-East High School",painter,
18,Beauford Delaney,male,1901-12-31T00:00:00Z,1979-03-25T00:00:00Z,Knoxville,14th arrondissement of Paris,,"Harvard University, Austin-East High School",painter,
19,Beauford Delaney,male,1901-12-30T00:00:00Z,1979-03-26T00:00:00Z,Knoxville,14th arrondissement of Paris,,"Harvard University, Austin-East High School",painter,
20,Beauford Delaney,male,1901-12-31T00:00:00Z,1979-03-26T00:00:00Z,Knoxville,14th arrondissement of Paris,,"Harvard University, Austin-East High School",painter,
34,Charles Wilbert White,male,1918-04-02T00:00:00Z,1979-10-03T00:00:00Z,Chicago,Los Angeles,,School of the Art Institute of Chicago,"artist, painter, printmaker",
35,Charles Wilbert White,male,1918-04-02T00:00:00Z,1979-11-03T00:00:00Z,Chicago,Los Angeles,,School of the Art Institute of Chicago,"artist, painter, printmaker",
69,Gertrude Morgan,female,1900-01-01T00:00:00Z,1980-07-08T00:00:00Z,LaFayette,New Orleans,"New Orleans, LaFayette, Columbus",,"poet, missionary, preacher, artist, musician, ...",
70,Gertrude Morgan,female,1900-04-07T00:00:00Z,1980-07-08T00:00:00Z,LaFayette,New Orleans,"New Orleans, LaFayette, Columbus",,"poet, missionary, preacher, artist, musician, ...",
79,Henry Ossawa Tanner,male,1859-06-21T00:00:00Z,1937-05-24T00:00:00Z,Pittsburgh,6th arrondissement of Paris,Henry O. Tanner House,"Académie Julian, Pennsylvania Academy of the F...","painter, university teacher, printmaker, photo...",Clark University
80,Henry Ossawa Tanner,male,1859-06-21T00:00:00Z,1937-05-25T00:00:00Z,Pittsburgh,6th arrondissement of Paris,Henry O. Tanner House,"Académie Julian, Pennsylvania Academy of the F...","photographer, painter, university teacher, pri...",Clark University


In [ ]:
dupe.to_csv(r'Dupe.csv', index = True)
from google.colab import files 

## Download Data as CSV

In [ ]:
dl = df.to_csv('Wikidata_{}.csv'.format(pd.datetime.now().strftime("%Y-%m-%d_%Hh%Mm%Ss")), index=True) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


In [ ]:
from google.colab import files
files.download('/content/Wikidata_2021-09-22 22h09m39s.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Geocode Data

https://towardsdatascience.com/pythons-geocoding-convert-a-list-of-addresses-into-a-map-f522ef513fd6

In [ ]:
!pip install geopandas
!pip install geopy

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="sample app")

In [ ]:
# 2- - create location column
df['location'] = df['birthplace'].apply(geolocator.geocode)

In [ ]:
df['location']

In [ ]:
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)